In [1]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo

import matplotlib.pyplot as plt                   # grafico
import matplotlib.dates as mdates                 # trabalhar datas

import statsmodels.api as sm                      # regressao
from statsmodels.api import add_constant          # para a regressao considerar o intercepto

import numpy as np
import scipy.stats

from statsmodels.tsa.stattools import coint       # Funcao Coint ()
from statsmodels.tsa.stattools import adfuller

# Declaracoes
pd.options.display.float_format = '{:.2f}'.format

#!pip install MetaTrader5
#!pip install --upgrade MetaTrader5
import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
import time
import pytz

import ipywidgets


### DECLARACAO DE FUNCOES ###

In [2]:

# Baixa base de dados e retorna uma base da forma [data,close_ativo_1, close_ativo_2]
def baixar_yahoo(tickers,inicio,fim,baixayahoo):

    # Percorre a lista tickers, lendo o yahoo e salvando no DataFrame db
    db = pd.DataFrame()
    if (baixayahoo == True):
        
        for i in tickers:
            cot = pdr.DataReader(i, 'yahoo', inicio,fim)
            cot['Ativo']=i
            db = pd.concat([db,cot],sort=True)  

        db.to_csv('Ativos.csv')
        #db.to_excel('ativos.xlsx')
    
    db = pd.read_csv('Ativos.csv',parse_dates=[0], infer_datetime_format = True)
    #db = pd.read_excel('ITSA4GOLL4.xlsx')
    db = db[['Ativo','Date','Open','High','Low','Adj Close']].rename(columns={'Adj Close': 'Close'})    
    
    # Cria Tabela com os fechamentos dos dois ativos [DATA, Close1, Close2]
    esquerda = db[db['Ativo']==tickers[0]].set_index('Date')['Close']
    direita = db[db['Ativo']==tickers[1]].set_index('Date')['Close']
    par = pd.merge(left=esquerda, right=direita, left_on='Date', right_on='Date').rename(columns={'Close_x': tickers[0], 'Close_y': tickers[1]})

    #par.to_excel('Par.xlsx')
    
    return par

# Baixa base de dados e retorna uma base da forma [data,close_ativo_1, close_ativo_2]
def Mt5_dados(tickers,inicio,fim):

    # conecte-se ao MetaTrader 5
    if not mt5.initialize():
       
        print("initialize() failed, error code =",mt5.last_error())
        mt5.shutdown()

    else:
   
        # imprimimos informações sobre o estado da conexão, o nome do servidor e a conta de negociação
        print(mt5.terminal_info())
        # imprimimos informações sobre a versão do MetaTrader 5
        print(mt5.version())
        
        #quebra as strings dates(inicio,fim) recebidas e transformando em integer
        inicio_= inicio.split('-')
        inicio = [int(element) for element in inicio_]
        fim_= fim.split('-')
        fim = [int(element) for element in fim_]

        timezone = pytz.timezone("Etc/UTC")

        # obtendo o par de acoes com base nas datas enviadas nas strings, definindo o timezone para nao interfir com o tisp do mt5
        stock1 = mt5.copy_rates_range(tickers[0], mt5.TIMEFRAME_D1, datetime(inicio[0],inicio[1],inicio[2],tzinfo=timezone),datetime(fim[0],fim[1],fim[2]))
        stock2 = mt5.copy_rates_range(tickers[1], mt5.TIMEFRAME_D1, datetime(inicio[0],inicio[1],inicio[2],tzinfo=timezone),datetime(fim[0],fim[1],fim[2]))
        
        # concluímos a conexão ao MetaTrader 5
        mt5.shutdown()

        esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
        direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})

        stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
        stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
        stock_df.drop('time',axis=1,inplace = True)

    return stock_df.set_index('Date')

In [3]:
def regressao_univariada(x, y):
    
    # Gera a REGRESSAO LINEAR UNIVARIADA -> y = b*x + c + e
    X = sm.add_constant(x)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = par_subset[tickers[0]]-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore 

def regressao_multivariada(x, y, periodo):

    # Gera a REGRESSAO LINEAR MULTIVARIADA (QuantGo "Simples") -> y = b*x + c*t + d + e
    X = np.column_stack((x, range(1,periodo+1,1)))
    X = sm.add_constant(X)
    #print(X)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = y-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore 


def regressao_residuos(residuos):
  
    residuos_shifted = residuos.diff(1).fillna(method="bfill")  
    delta = residuos - residuos_shifted
    print(residuos_shifted)
    X = sm.add_constant(residuos_shifted, prepend=True)
    res = sm.OLS(residuos,X).fit()
    print(res.summary)
    return res.params[1]

# ADF da Statsmodel
def adftest(df):

    adf  = adfuller(df,maxlag=1, autolag="BIC")
    
    print('\n')
    print (adf)
    
    if   adf[0] < adf[4]['1%']:  adfc='99%'
    elif adf[0] < adf[4]['5%']:  adfc='95%'
    elif adf[0] < adf[4]['10%']: adfc='90%'
    else:                        adfc='-'                 

    if   adf[0] < -4.32:  adfc_2='99%'
    elif adf[0] < -3.67:  adfc_2='95%'
    elif adf[0] < -3.28: adfc_2='90%'
    else:                        adfc_2='-'                 
        
    if   adf[0] < -3.58:  adfc_3='99%'
    elif adf[0] < -3.22:  adfc_3='95%'
    elif adf[0] < -2.6: adfc_3='90%'
    else:                        adfc_3='-'                 
    

    
    return adf[0], adfc,adf[1],adfc_2,adfc_3      

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''
    
    r, p = scipy.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = scipy.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    
    return r, p, lo, hi
    
def pct_financeiro(x,y,coef_ang,residuo):
    
    ultimo_x = x.tail(1)[0]
    ultimo_y = y.tail(1)[0]
    
    fin_x = ultimo_x*coef_ang
    fin_y = ultimo_y
    
    if (residuo > 0):
        compra = fin_x
        venda = fin_y
    else:
        compra = fin_y
        venda = fin_x
 
    cv = "{:.0%}".format(compra/venda)
    return cv

def calculo_meia_vida(residuo):
    # Retirado da planilha do Ferro
    coef_ang_residuos = regressao_residuos(residuo) 
    print(coef_ang_residuos)
    mv_beta = -1*np.log(1+coef_ang_residuos)
    meia_vida = 2/mv_beta  
    return meia_vida

def calculo_meia_vida1(residuo):
    # Retirado de um paper e adaptado pela formula da planilha do Ferro
    price = pd.Series(residuo)  
    lagged_price = price.shift(1).fillna(method="bfill")  
    delta = price - lagged_price  
    beta = np.polyfit(lagged_price, delta, 1)[0] 
    #half_life = ((-2*np.log(2))/beta)  # paper (varios), mas sem o 2*
    half_life = 2/(-1*np.log(1+beta))   # planilha ferro
     
    return "{} Dias".format(int(round(half_life)))

In [4]:
## Funcoes graficas

def grafico_residuo(residuo):
        
    plt.figure(figsize=(20,4))
    ax = residuo.plot(color='g', grid=True, label='Z-Score')
    plt.axhline(residuo.mean(), color='red')
    plt.axhline(residuo.mean()+2*residuo.std(), color='blue')
    plt.axhline(residuo.mean()-2*residuo.std(), color='blue')

    ax.xaxis_date()  # formata o timestamp para o formato data
    ax.set_axisbelow(True)
    ax.set_title('Residuos', color='black')
    ax.set_facecolor('white')
    ax.figure.set_facecolor('white')

    plt.legend()
    plt.show()    
        

In [29]:

#tickers = ['CMIG3','ECOR3'];
tickers = ['QUAL3','ALPA4'];
#tickers = ['ALPA4','ENEV3'];

# Busca cotacoes no Yahoo Finance
par = Mt5_dados(tickers, '2019-1-1','2020-7-31')
#par = baixar_yahoo(tickers, '2019-1-1','2020-7-28',True)
par.to_excel('ativos.xlsx')
par['ratio']  = par[tickers[0]]/par[tickers[1]]
par['ratio1'] = par[tickers[1]]/par[tickers[0]]


# Cria o cubo de periodos
cubo = pd.DataFrame(columns=['Periodo','Dickey_Fuller','ADF','ADF_2','ADF_3','Coef_Ang','QTD_Desvios','Fisher_min','Fisher_max'
                             ,'ADF_pvalue','pct_fin','meia_vida'
                             ,#'coint_p', 'coint_dickey','coint_adf'
                            ])

# Popula o cubo de periodos
#for i in range(100,260,10):
for i in [100,120,140,160,180,200,220,240,250]:
 
    # Cria subset do tamanho do periodo atual da iteração e seta a variavel indep (x) e dependente (y)
    par_subset = par.tail(i)
    x = par_subset[tickers[1]]
    y = par_subset[tickers[0]]
   
    # Adiciona o PERIODO analisado no dataframe
    cubo = cubo.append({'Periodo': int(i)}, ignore_index=True)
    
    # Gera a REGRESSAO e retorna o Coef Ang, e o z-score
    coef_ang, residuo, zscore    = regressao_multivariada(x, y, i)

    # Teste de Estacionariedade dos Resíduos (ADF)
    adfstat,adfc,adfpvalue,adfc_2,adfc_3 = adftest(residuo)
 
    # Calculo do Fisher
    ficher_r, fisher_pvalue, fisher_lo, fisher_hi = pearsonr_ci(x.pct_change().fillna(method="bfill"), y.pct_change().fillna(method="bfill"))
    
    # Calculo do % Financeiro (C/V)
    pct_fin = pct_financeiro(x,y,coef_ang, zscore.tail(1).values)
    
    # Calculo da Meia-vida - ORNSTEIN-UHLENBECK
    meia_vida = calculo_meia_vida1(residuo)
    
    # Preenchimento do cubo
    cubo.loc[cubo['Periodo']==i, 'Coef_Ang'     ] = coef_ang 
    cubo.loc[cubo['Periodo']==i, 'QTD_Desvios'  ] = zscore.tail(1)[0]
    cubo.loc[cubo['Periodo']==i, 'Dickey_Fuller'] = adfstat
    cubo.loc[cubo['Periodo']==i, 'ADF'          ] = adfc 
    cubo.loc[cubo['Periodo']==i, 'ADF_pvalue'   ] = adfpvalue 
    cubo.loc[cubo['Periodo']==i, 'Fisher_min'   ] = "{:.0%}".format(fisher_lo)
    cubo.loc[cubo['Periodo']==i, 'Fisher_max'   ] = "{:.0%}".format(fisher_hi)
    cubo.loc[cubo['Periodo']==i, 'pct_fin'      ] = pct_fin
    cubo.loc[cubo['Periodo']==i, 'meia_vida'    ] = meia_vida
    cubo.loc[cubo['Periodo']==i, 'ADF_2'        ] = adfc_2
    cubo.loc[cubo['Periodo']==i, 'ADF_3'        ] = adfc_3



    # Coint - apenas para comparacao
    #score, pvalue, _ = coint(residuo[1:],residuo.diff()[1:], maxlag=1, autolag='t-stat')

    #if (score < _[0]): coint_adf='99%'
    #elif (score < _[1]): coint_adf='95%'
    #elif (score < _[2]): coint_adf='90%'
    #else: coint_adf='0%'
    #cubo.loc[cubo['Periodo']==i, 'coint_dickey']    = score
    #cubo.loc[cubo['Periodo']==i, 'coint_p']       = pvalue
    #cubo.loc[cubo['Periodo']==i, 'coint_adf']     = coint_adf
    
#par.to_excel("Par.xlsx")    
display(cubo)

# Desvio médio do Coef. Angular - apenas dos periodos cointegrados
cubo_cointegrado = cubo.loc[cubo['ADF'] != '-']
print("Desvio Medio do Coef. Angular: {:.2%}" .format(cubo_cointegrado['Coef_Ang'].std()/cubo_cointegrado['Coef_Ang'].mean()))

print()


# Selecionar o periodo para mostrar o gráfico dos residuos
@ipywidgets.interact(Periodo=(["Selecione o Período", 100,120,140,160,180,200,220,240,250]))
def Grafico(Periodo):
    if (Periodo != "Selecione o Período"):  
        coef_ang, residuo, zscore = regressao_multivariada(par[tickers[1]].tail(Periodo), par[tickers[0]].tail(Periodo),Periodo)
        grafico_residuo(zscore)
        print(coef_ang, residuo, zscore)

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2560, maxbars=100000, codepage=1252, ping_last=13931, community_balance=0.0, retransmission=0.06527924616661818, company='Rico (XP Investimentos CCTVM)', name='Rico - MetaTrader 5', language='Portuguese (Brazil)', path='C:\\Program Files\\Rico - MetaTrader 5', ...)
(500, 2560, '24 Jul 2020')


(-4.499343461655018, 0.00019717773314503116, 0, 99, {'1%': -3.498198082189098, '5%': -2.891208211860468, '10%': -2.5825959973472097}, 296.64824674662503)


(-5.889978451983369, 2.9400406988644105e-07, 0, 119, {'1%': -3.4865346059036564, '5%': -2.8861509858476264, '10%': -2.579896092790057}, 398.25911013827533)


(-5.821351281495043, 4.1758013869346993e-07, 0, 139, {'1%': -3.4782936965183815, '5%': -2.882567574015525, '10%': -2.5779830117488745}, 474.33834546399333)


,Periodo,Dickey_Fuller,ADF,ADF_2,ADF_3,Coef_Ang,QTD_Desvios,Fisher_min,Fisher_max,ADF_pvalue,pct_fin,meia_vida
0,100.00,-4.50,99%,99%,99%,1.19,-1.91,49%,73%,0.00,75%,5 Dias
1,120.00,-5.89,99%,99%,99%,1.50,-1.83,56%,76%,0.00,59%,3 Dias
2,140.00,-5.82,99%,99%,99%,1.54,-1.63,55%,74%,0.00,58%,4 Dias
3,160.00,-5.29,99%,99%,99%,1.54,-1.69,53%,72%,0.00,58%,6 Dias
4,180.00,-5.53,99%,99%,99%,1.54,-1.77,54%,71%,0.00,58%,6 Dias
5,200.00,-5.82,99%,99%,99%,1.54,-1.80,53%,70%,0.00,58%,6 Dias
6,220.00,-6.13,99%,99%,99%,1.58,-2.05,52%,69%,0.00,56%,6 Dias
7,240.00,-6.46,99%,99%,99%,1.60,-2.20,53%,68%,0.00,56%,6 Dias
8,250.00,-6.67,99%,99%,99%,1.62,-2.27,46%,63%,0.00,55%,5 Dias


Desvio Medio do Coef. Angular: 8.53%



interactive(children=(Dropdown(description='Periodo', options=('Selecione o Período', 100, 120, 140, 160, 180,…

Atencao

- Calculo da meia vida: o calculo que peguei da planilha difere com os que encontrei na internet. Nenhum deles batia, fiz um mix dos dois via tentativa e erro e agora está bem proximo.. mas acho que temos que entender melhor

- Fisher: baixei uma formula, nao entendi nada.. tbm seria legal entender

- ADF: estou testando de tudo, o %ADF nao bate nem a pau.. coloquei o ADF2 e o ADF3 que estao na planilha do ferro.. mas ele alterna entre os dois de uma forma q nao entendi, baseada na importancia do coeficiente do tempo na regressao.. e mesmo assim parece q nao bateria.. apesar de que para 'CMIG3.SA','ECOR3.SA' bateu top

